In [11]:
import pandas as pd
import numpy as np
import csv
import xlsxwriter
from csv import reader, writer
from openpyxl import Workbook
import datetime as dt

In [12]:
#Import mobile route info
cs2 = "z_MAY_ROUTES.csv"
mobile = pd.read_csv(cs2)
mobile.head(2)

,shift_date,id,driver,truck,ship_to,customer,address,city_st_zip,type,fills,total_time,product,margin,gallons,fee,profit,paylocity_time
0,5/1/2022,D0000223782,Mason Valenzuela,JUB252,73215_1,Swire CC - Wilsonville,9750 SW Barber St,"WILSONVILLE, OR 97070",FLEET,54.0,2.600000,68,0.299,1396,0,417.404,12.3
1,5/1/2022,D0000223780,Mason Valenzuela,JUB252,73320_1,Swire - Yard Goat,9750 SW Barber Street,"WILSONVILLE, OR 97070",FLEET,1.0,0.133333,68,0.240,8,0,1.920,12.3


In [13]:
#Zones & Messages > Customer Visits > Select All assets > View by Asset
#Pull Details Advanced Report
#Unhide Data Tab
#Delete header info from Data tab
#Change column Q duration to 00:00:00 Time format
#Save as CSV
cs1 = "customer_visitsz.csv"
df = pd.read_csv(cs1)
df.head(2)

,.Device.DeviceName,.Device.DeviceId,.Device.DeviceComment,.Device.DeviceGroup,.Driver.UserFirstName,.Driver.UserLastName,.Driver.UserName,.Driver.UserId,.Driver.UserComment,.Driver.DriverGroup,.ExceptionRule.ExceptionRuleName,.ExceptionRule.ExceptionRuleId,.ExceptionRule.ExceptionRuleComment,.StopZone.ZoneZoneTypes,.StopZone.ZoneExternalReference,ExceptionDetailStartTime,ExceptionDuration,ExceptionDetailLongitude,ExceptionDetailLatitude,ExceptionTripDistance
0,JUB001 (DEF),b6,Peterbilt PB330 2004,DEF,Craig,Herzog,cherzog,b1F,NaN,Entire Organization,Jubitz Truck Service Center,a9LJJYcN350GboybeOMBKTA,NaN,Customer Zone,NaN,28/04/2022 10:38:02 AM,19:54:00,-122.671082,45.597237,9.866968
1,JUB001 (DEF),b6,Peterbilt PB330 2004,DEF,Craig,Herzog,cherzog,b1F,NaN,Entire Organization,Jubitz Truck Stop,aUOIWBvpiTEiqiO52haf_Rw,NaN,Customer Zone,10000_20,2/5/2022 6:32,0:09:00,-122.667725,45.596954,9.866968


In [14]:
df2 = df.drop(columns=['.Device.DeviceId', '.Device.DeviceComment', '.Device.DeviceGroup', '.Driver.UserFirstName', '.Driver.UserLastName', '.Driver.UserId', '.Driver.UserComment', '.Driver.DriverGroup', '.ExceptionRule.ExceptionRuleId', '.ExceptionRule.ExceptionRuleComment', '.StopZone.ZoneZoneTypes', 'ExceptionDetailLongitude', 'ExceptionDetailLatitude'])
df2.head(2)

,.Device.DeviceName,.Driver.UserName,.ExceptionRule.ExceptionRuleName,.StopZone.ZoneExternalReference,ExceptionDetailStartTime,ExceptionDuration,ExceptionTripDistance
0,JUB001 (DEF),cherzog,Jubitz Truck Service Center,NaN,28/04/2022 10:38:02 AM,19:54:00,9.866968
1,JUB001 (DEF),cherzog,Jubitz Truck Stop,10000_20,2/5/2022 6:32,0:09:00,9.866968


In [15]:
df2.rename({'.Device.DeviceName':'truck', '.Driver.UserName':'driver', '.ExceptionRule.ExceptionRuleName':'zone', '.StopZone.ZoneExternalReference':'ship_to', 'ExceptionTripDistance':'distance', 'ExceptionDetailStartTime':'arrived', 'ExceptionDuration':'duration'},axis =1, inplace = True)
df2.head(1)

,truck,driver,zone,ship_to,arrived,duration,distance
0,JUB001 (DEF),cherzog,Jubitz Truck Service Center,NaN,28/04/2022 10:38:02 AM,19:54:00,9.866968


In [16]:
df2['date'] = pd.to_datetime(df2['arrived']).dt.date
df2.head()

,truck,driver,zone,ship_to,arrived,duration,distance,date
0,JUB001 (DEF),cherzog,Jubitz Truck Service Center,NaN,28/04/2022 10:38:02 AM,19:54:00,9.866968,2022-04-28
1,JUB001 (DEF),cherzog,Jubitz Truck Stop,10000_20,2/5/2022 6:32,0:09:00,9.866968,2022-02-05
2,JUB001 (DEF),cherzog,Ps Trucking-Def,553966_6,2/5/2022 6:55,0:09:00,6.046319,2022-02-05
3,JUB001 (DEF),cherzog,Bob's Metals - Portland,72168_1,2/5/2022 7:20,0:05:00,6.461678,2022-02-05
4,JUB001 (DEF),cherzog,Goodfellow Bros Yard,527560_2,2/5/2022 7:43,0:06:00,5.293309,2022-02-05


In [17]:
df3 = df2.drop(columns=['arrived'])
df3.head(2)

,truck,driver,zone,ship_to,duration,distance,date
0,JUB001 (DEF),cherzog,Jubitz Truck Service Center,NaN,19:54:00,9.866968,2022-04-28
1,JUB001 (DEF),cherzog,Jubitz Truck Stop,10000_20,0:09:00,9.866968,2022-02-05


In [18]:
df3['duration'] = pd.to_timedelta(df3['duration'])
df3.head(2)

,truck,driver,zone,ship_to,duration,distance,date
0,JUB001 (DEF),cherzog,Jubitz Truck Service Center,NaN,19:54:00,9.866968,2022-04-28
1,JUB001 (DEF),cherzog,Jubitz Truck Stop,10000_20,00:09:00,9.866968,2022-02-05


In [19]:
df3['day_hours'] = df3['duration'].dt.days * 24
df3.head()

,truck,driver,zone,ship_to,duration,distance,date,day_hours
0,JUB001 (DEF),cherzog,Jubitz Truck Service Center,NaN,19:54:00,9.866968,2022-04-28,0
1,JUB001 (DEF),cherzog,Jubitz Truck Stop,10000_20,00:09:00,9.866968,2022-02-05,0
2,JUB001 (DEF),cherzog,Ps Trucking-Def,553966_6,00:09:00,6.046319,2022-02-05,0
3,JUB001 (DEF),cherzog,Bob's Metals - Portland,72168_1,00:05:00,6.461678,2022-02-05,0
4,JUB001 (DEF),cherzog,Goodfellow Bros Yard,527560_2,00:06:00,5.293309,2022-02-05,0


In [20]:
df3['hours'] = df3['duration'].dt.components['hours']
df3.head()

,truck,driver,zone,ship_to,duration,distance,date,day_hours,hours
0,JUB001 (DEF),cherzog,Jubitz Truck Service Center,NaN,19:54:00,9.866968,2022-04-28,0,19
1,JUB001 (DEF),cherzog,Jubitz Truck Stop,10000_20,00:09:00,9.866968,2022-02-05,0,0
2,JUB001 (DEF),cherzog,Ps Trucking-Def,553966_6,00:09:00,6.046319,2022-02-05,0,0
3,JUB001 (DEF),cherzog,Bob's Metals - Portland,72168_1,00:05:00,6.461678,2022-02-05,0,0
4,JUB001 (DEF),cherzog,Goodfellow Bros Yard,527560_2,00:06:00,5.293309,2022-02-05,0,0


In [21]:
df3['minutes'] = df3['duration'].dt.components['minutes'] / 60
df3.head()

,truck,driver,zone,ship_to,duration,distance,date,day_hours,hours,minutes
0,JUB001 (DEF),cherzog,Jubitz Truck Service Center,NaN,19:54:00,9.866968,2022-04-28,0,19,0.900000
1,JUB001 (DEF),cherzog,Jubitz Truck Stop,10000_20,00:09:00,9.866968,2022-02-05,0,0,0.150000
2,JUB001 (DEF),cherzog,Ps Trucking-Def,553966_6,00:09:00,6.046319,2022-02-05,0,0,0.150000
3,JUB001 (DEF),cherzog,Bob's Metals - Portland,72168_1,00:05:00,6.461678,2022-02-05,0,0,0.083333
4,JUB001 (DEF),cherzog,Goodfellow Bros Yard,527560_2,00:06:00,5.293309,2022-02-05,0,0,0.100000


In [22]:
df3['total_hours'] = df3['day_hours'] + df3['hours'] + df3['minutes']
df3.head()

,truck,driver,zone,ship_to,duration,distance,date,day_hours,hours,minutes,total_hours
0,JUB001 (DEF),cherzog,Jubitz Truck Service Center,NaN,19:54:00,9.866968,2022-04-28,0,19,0.900000,19.900000
1,JUB001 (DEF),cherzog,Jubitz Truck Stop,10000_20,00:09:00,9.866968,2022-02-05,0,0,0.150000,0.150000
2,JUB001 (DEF),cherzog,Ps Trucking-Def,553966_6,00:09:00,6.046319,2022-02-05,0,0,0.150000,0.150000
3,JUB001 (DEF),cherzog,Bob's Metals - Portland,72168_1,00:05:00,6.461678,2022-02-05,0,0,0.083333,0.083333
4,JUB001 (DEF),cherzog,Goodfellow Bros Yard,527560_2,00:06:00,5.293309,2022-02-05,0,0,0.100000,0.100000


In [23]:
df4 = df3.drop(columns=['duration', 'day_hours', 'hours', 'minutes'])
df4.head(2)

,truck,driver,zone,ship_to,distance,date,total_hours
0,JUB001 (DEF),cherzog,Jubitz Truck Service Center,NaN,9.866968,2022-04-28,19.90
1,JUB001 (DEF),cherzog,Jubitz Truck Stop,10000_20,9.866968,2022-02-05,0.15


In [24]:
df5 = df4.groupby(['date', 'ship_to', 'zone', 'driver', 'truck'], as_index = False).sum()
df5.head(10)

,date,ship_to,zone,driver,truck,distance,total_hours
0,2022-01-05,10000_20,Jubitz Truck Stop,mvalenzuela,JUB252,33.819160,0.983333
1,2022-01-05,15,Jubitz Gas Station,mvalenzuela,JUB252,0.195620,0.550000
2,2022-01-05,72403_1,Sysco - Wilsonville,mvalenzuela,JUB252,3.951233,4.250000
3,2022-01-05,73215_1,Swire Yard Goat,mvalenzuela,JUB252,25.888205,2.700000
4,2022-02-05,10000_10,Sandy/Fairview Cardlock (JUB340),khaebe,TNT61,18.301695,0.516667
5,2022-02-05,10000_17,Sequential - 17,dcooper,TNT8,28.779536,0.966667
6,2022-02-05,10000_19,Aurora Cardlock (AMO),dcooper,TNT8,0.044281,0.350000
7,2022-02-05,10000_20,Jubitz Truck Stop,cherzog,JUB001 (DEF),9.866968,0.150000
8,2022-02-05,10000_20,Jubitz Truck Stop,dcooper,TNT8,89.007439,1.916667
9,2022-02-05,10000_20,Jubitz Truck Stop,dyambra,JUB233,10.632473,0.366667


In [25]:
#df4.to_csv("customer_visit_detail.csv", header=True, index=False)

# ----------

## Summary

In [26]:
summary = df5.groupby(['ship_to', 'zone']).agg({'distance':['mean', 'min', 'max'], 'total_hours':['mean', 'min', 'max']})
summary.head(10)

distance                         \
                                                 mean        min         max   
ship_to   zone                                                                 
10000_10  Sandy/Fairview Cardlock (JUB340)  21.863289   1.860239   60.975647   
10000_17  Sequential - 17                   40.321150  28.759638   86.485081   
10000_181 Wilsonville Cardlock (701181)     21.444609   3.451225   31.130758   
10000_19  Aurora Cardlock (AMO)             24.402582   0.043866   67.344339   
10000_2   ILBI/Loy Clark Cardlock (A5N)      8.303987   8.303987    8.303987   
10000_20  Jubitz Truck Stop                 21.872948   0.041629  159.422751   
10000_21  Mccall - 21                       22.383138   9.407011  102.950604   
10000_23  Nustar Energy - 23                22.293180   5.692459   71.742255   
10000_271 Cornelius Cardlock (700271)       20.830543   0.123607   31.209067   
10000_272 Tualatin Cardlock (700272)        17.610055   1.819729   28.323475   

                                           total_hours                       
                                                  mean       min        max  
ship_to   zone                                                               
10000_10  Sandy/Fairview Cardlock (JUB340)    0.530769  0.016667   1.833333  
10000_17  Sequential - 17                     0.976667  0.533333   1.250000  
10000_181 Wilsonville Cardlock (701181)       1.246296  0.583333   2.016667  
10000_19  Aurora Cardlock (AMO)               0.639394  0.350000   1.000000  
10000_2   ILBI/Loy Clark Cardlock (A5N)       0.116667  0.116667   0.116667  
10000_20  Jubitz Truck Stop                   1.406354  0.000000  24.033333  
10000_21  Mccall - 21                         0.595370  0.266667   1.200000  
10000_23  Nustar Energy - 23                  0.751961  0.183333   2.250000  
10000_271 Cornelius Cardlock (700271)         1.422222  1.366667   1.466667  
10000_272 Tualatin Cardlock (700272)          1.025000  0.483333   1.266667

In [27]:
#summary.to_csv("customer_summary.csv")

In [28]:
with pd.ExcelWriter('customer_visits.xlsx', engine='xlsxwriter') as writer:    
    #Write each dataframe to a different worksheet.
    df4.to_excel(writer, sheet_name='Customer Visit Detail', header=True, index=False)
    summary.to_excel(writer, sheet_name='Customer Visit Summary', header=True, index=True)

# --------------

## Merge GeoTab and SLX/Trinium info

In [29]:
mobile.head(1)

,shift_date,id,driver,truck,ship_to,customer,address,city_st_zip,type,fills,total_time,product,margin,gallons,fee,profit,paylocity_time
0,5/1/2022,D0000223782,Mason Valenzuela,JUB252,73215_1,Swire CC - Wilsonville,9750 SW Barber St,"WILSONVILLE, OR 97070",FLEET,54.0,2.6,68,0.299,1396,0,417.404,12.3


In [30]:
mobile2 = mobile.drop(columns=['type', 'total_time', 'paylocity_time'])
mobile2.head(1)

,shift_date,id,driver,truck,ship_to,customer,address,city_st_zip,fills,product,margin,gallons,fee,profit
0,5/1/2022,D0000223782,Mason Valenzuela,JUB252,73215_1,Swire CC - Wilsonville,9750 SW Barber St,"WILSONVILLE, OR 97070",54.0,68,0.299,1396,0,417.404


In [31]:
mobile3 = mobile2.rename(columns={'shift_date':'date', 'id':'slx_id', 'driver':'slx_driver'})
mobile3.head(1)

,date,slx_id,slx_driver,truck,ship_to,customer,address,city_st_zip,fills,product,margin,gallons,fee,profit
0,5/1/2022,D0000223782,Mason Valenzuela,JUB252,73215_1,Swire CC - Wilsonville,9750 SW Barber St,"WILSONVILLE, OR 97070",54.0,68,0.299,1396,0,417.404


In [32]:
mobile3['date'] = pd.to_datetime(mobile3['date']).dt.date
mobile3.head(2)

,date,slx_id,slx_driver,truck,ship_to,customer,address,city_st_zip,fills,product,margin,gallons,fee,profit
0,2022-05-01,D0000223782,Mason Valenzuela,JUB252,73215_1,Swire CC - Wilsonville,9750 SW Barber St,"WILSONVILLE, OR 97070",54.0,68,0.299,1396,0,417.404
1,2022-05-01,D0000223780,Mason Valenzuela,JUB252,73320_1,Swire - Yard Goat,9750 SW Barber Street,"WILSONVILLE, OR 97070",1.0,68,0.240,8,0,1.920


In [33]:
merged = pd.merge(mobile3, df5, on=['date', 'ship_to', 'truck'], how='left')
merged.head()

,date,slx_id,slx_driver,truck,ship_to,customer,address,city_st_zip,fills,product,margin,gallons,fee,profit,zone,driver,distance,total_hours
0,2022-05-01,D0000223782,Mason Valenzuela,JUB252,73215_1,Swire CC - Wilsonville,9750 SW Barber St,"WILSONVILLE, OR 97070",54.0,68,0.2990,1396,0,417.404,NaN,NaN,NaN,NaN
1,2022-05-01,D0000223780,Mason Valenzuela,JUB252,73320_1,Swire - Yard Goat,9750 SW Barber Street,"WILSONVILLE, OR 97070",1.0,68,0.2400,8,0,1.920,NaN,NaN,NaN,NaN
2,2022-05-01,D0000223778,Mason Valenzuela,JUB252,72403_1,Sysco PDX-Mansfield-Wilsonvill,26250 SW Parkway CTR Dr,"WILSONVILLE, OR 97070",91.0,82,0.2750,2280,0,627.000,NaN,NaN,NaN,NaN
3,2022-05-02,D0000223701,DOUG BOWERMAN,JUB002,501402_5,American Cruise-QOW-Dalles,QUEEN OF THE WEST,"THE DALLES, OR 97058",1.0,28,0.4315,0,0,0.000,NaN,NaN,NaN,NaN
4,2022-05-02,D0000223701,DOUG BOWERMAN,JUB002,501402_5,American Cruise-QOW-Dalles,QUEEN OF THE WEST,"THE DALLES, OR 97058",1.0,28,0.4315,1950,0,841.425,NaN,NaN,NaN,NaN


In [34]:
#Drop service trucks
merged2 = merged[~merged.truck.str.contains('Service')]
merged2.head()

,date,slx_id,slx_driver,truck,ship_to,customer,address,city_st_zip,fills,product,margin,gallons,fee,profit,zone,driver,distance,total_hours
0,2022-05-01,D0000223782,Mason Valenzuela,JUB252,73215_1,Swire CC - Wilsonville,9750 SW Barber St,"WILSONVILLE, OR 97070",54.0,68,0.2990,1396,0,417.404,NaN,NaN,NaN,NaN
1,2022-05-01,D0000223780,Mason Valenzuela,JUB252,73320_1,Swire - Yard Goat,9750 SW Barber Street,"WILSONVILLE, OR 97070",1.0,68,0.2400,8,0,1.920,NaN,NaN,NaN,NaN
2,2022-05-01,D0000223778,Mason Valenzuela,JUB252,72403_1,Sysco PDX-Mansfield-Wilsonvill,26250 SW Parkway CTR Dr,"WILSONVILLE, OR 97070",91.0,82,0.2750,2280,0,627.000,NaN,NaN,NaN,NaN
3,2022-05-02,D0000223701,DOUG BOWERMAN,JUB002,501402_5,American Cruise-QOW-Dalles,QUEEN OF THE WEST,"THE DALLES, OR 97058",1.0,28,0.4315,0,0,0.000,NaN,NaN,NaN,NaN
4,2022-05-02,D0000223701,DOUG BOWERMAN,JUB002,501402_5,American Cruise-QOW-Dalles,QUEEN OF THE WEST,"THE DALLES, OR 97058",1.0,28,0.4315,1950,0,841.425,NaN,NaN,NaN,NaN


In [35]:
#Drop internal sites
merged3 = merged2[~merged2.ship_to.str.startswith('10000')]
merged3.head(2)

,date,slx_id,slx_driver,truck,ship_to,customer,address,city_st_zip,fills,product,margin,gallons,fee,profit,zone,driver,distance,total_hours
0,2022-05-01,D0000223782,Mason Valenzuela,JUB252,73215_1,Swire CC - Wilsonville,9750 SW Barber St,"WILSONVILLE, OR 97070",54.0,68,0.299,1396,0,417.404,NaN,NaN,NaN,NaN
1,2022-05-01,D0000223780,Mason Valenzuela,JUB252,73320_1,Swire - Yard Goat,9750 SW Barber Street,"WILSONVILLE, OR 97070",1.0,68,0.240,8,0,1.920,NaN,NaN,NaN,NaN


In [36]:
merged4 = merged3[merged3['slx_id'].notna()]
merged4.head()

,date,slx_id,slx_driver,truck,ship_to,customer,address,city_st_zip,fills,product,margin,gallons,fee,profit,zone,driver,distance,total_hours
0,2022-05-01,D0000223782,Mason Valenzuela,JUB252,73215_1,Swire CC - Wilsonville,9750 SW Barber St,"WILSONVILLE, OR 97070",54.0,68,0.2990,1396,0,417.404,NaN,NaN,NaN,NaN
1,2022-05-01,D0000223780,Mason Valenzuela,JUB252,73320_1,Swire - Yard Goat,9750 SW Barber Street,"WILSONVILLE, OR 97070",1.0,68,0.2400,8,0,1.920,NaN,NaN,NaN,NaN
2,2022-05-01,D0000223778,Mason Valenzuela,JUB252,72403_1,Sysco PDX-Mansfield-Wilsonvill,26250 SW Parkway CTR Dr,"WILSONVILLE, OR 97070",91.0,82,0.2750,2280,0,627.000,NaN,NaN,NaN,NaN
3,2022-05-02,D0000223701,DOUG BOWERMAN,JUB002,501402_5,American Cruise-QOW-Dalles,QUEEN OF THE WEST,"THE DALLES, OR 97058",1.0,28,0.4315,0,0,0.000,NaN,NaN,NaN,NaN
4,2022-05-02,D0000223701,DOUG BOWERMAN,JUB002,501402_5,American Cruise-QOW-Dalles,QUEEN OF THE WEST,"THE DALLES, OR 97058",1.0,28,0.4315,1950,0,841.425,NaN,NaN,NaN,NaN


In [37]:
merged4['profit_per_hour'] = merged4['profit']/merged4['total_hours']
merged4.head(3)


,date,slx_id,slx_driver,truck,ship_to,customer,address,city_st_zip,fills,product,margin,gallons,fee,profit,zone,driver,distance,total_hours,profit_per_hour
0,2022-05-01,D0000223782,Mason Valenzuela,JUB252,73215_1,Swire CC - Wilsonville,9750 SW Barber St,"WILSONVILLE, OR 97070",54.0,68,0.299,1396,0,417.404,NaN,NaN,NaN,NaN,NaN
1,2022-05-01,D0000223780,Mason Valenzuela,JUB252,73320_1,Swire - Yard Goat,9750 SW Barber Street,"WILSONVILLE, OR 97070",1.0,68,0.240,8,0,1.920,NaN,NaN,NaN,NaN,NaN
2,2022-05-01,D0000223778,Mason Valenzuela,JUB252,72403_1,Sysco PDX-Mansfield-Wilsonvill,26250 SW Parkway CTR Dr,"WILSONVILLE, OR 97070",91.0,82,0.275,2280,0,627.000,NaN,NaN,NaN,NaN,NaN


In [38]:
smart = merged4.groupby(['ship_to', 'zone']).agg({'ship_to':['count'],'distance':['min','mean', 'max'],'profit_per_hour':['min','mean', 'max'],'margin':['min','mean', 'max'],'gallons':['min','mean', 'max'], 'total_hours':['min','mean', 'max']})
smart.head(10)

ship_to   distance             \
                                            count        min       mean   
ship_to  zone                                                             
15       Jubitz Gas Station                     4  10.034814  10.522479   
381031_1 Eastside Paving Inc                    1  14.980039  14.980039   
40750_1  Morton Tree Srvc-West Linn             1   9.706425   9.706425   
509188_1 C & R Tractor & Landscaping            5   5.673559  18.269507   
514125_2 Western Forest Prod-Fruit Valley       1   2.248133   2.248133   
515550_1 Staples - Main                         2   3.268667   3.292146   
515566_2 Corwin Beverage-Ridgefield             2   9.243543  12.571997   
515955_1 Djs Elec                               2   7.425428  13.637506   
527560_2 Goodfellow Bros Yard                   1   8.547404   8.547404   
527605_1 Gould Sunrise                          1  15.619465  15.619465   

                                                     profit_per_hour  \
                                                 max             min   
ship_to  zone                                                          
15       Jubitz Gas Station                11.978386       90.610169   
381031_1 Eastside Paving Inc               14.980039      401.818182   
40750_1  Morton Tree Srvc-West Linn         9.706425      499.200000   
509188_1 C & R Tractor & Landscaping       36.786301      348.750000   
514125_2 Western Forest Prod-Fruit Valley   2.248133      698.400000   
515550_1 Staples - Main                     3.315624      160.047458   
515566_2 Corwin Beverage-Ridgefield        15.900451      121.987013   
515955_1 Djs Elec                          19.849585      416.666667   
527560_2 Goodfellow Bros Yard               8.547404      530.434286   
527605_1 Gould Sunrise                     15.619465      975.857143   

                                                                   margin  \
                                                 mean          max    min   
ship_to  zone                                                               
15       Jubitz Gas Station                189.358719   297.000000  0.027   
381031_1 Eastside Paving Inc               401.818182   401.818182  0.500   
40750_1  Morton Tree Srvc-West Linn        499.200000   499.200000  0.650   
509188_1 C & R Tractor & Landscaping       555.937500  1006.500000  0.350   
514125_2 Western Forest Prod-Fruit Valley  698.400000   698.400000  0.440   
515550_1 Staples - Main                    164.506082   168.964706  0.430   
515566_2 Corwin Beverage-Ridgefield        161.271767   200.556522  0.310   
515955_1 Djs Elec                          528.645833   640.625000  0.250   
527560_2 Goodfellow Bros Yard              530.434286   530.434286  0.180   
527605_1 Gould Sunrise                     975.857143   975.857143  0.550   

                                                        gallons           \
                                            mean    max     min     mean   
ship_to  zone                                                              
15       Jubitz Gas Station                0.027  0.027    3300  5400.25   
381031_1 Eastside Paving Inc               0.500  0.500     442   442.00   
40750_1  Morton Tree Srvc-West Linn        0.650  0.650     128   128.00   
509188_1 C & R Tractor & Landscaping       0.350  0.350     465   645.80   
514125_2 Western Forest Prod-Fruit Valley  0.440  0.440     291   291.00   
515550_1 Staples - Main                    0.430  0.430     334   350.00   
515566_2 Corwin Beverage-Ridgefield        0.310  0.310     496   500.50   
515955_1 Djs Elec                          0.250  0.250    1000  1012.50   
527560_2 Goodfellow Bros Yard              0.180  0.180    1719  1719.00   
527605_1 Gould Sunrise                     0.550  0.550     414   414.00   

                                                total_hours            \
                                            max         min      

In [39]:
with pd.ExcelWriter('Geotab_SLX_Report.xlsx', engine='xlsxwriter') as writer:    
    #Write each dataframe to a different worksheet.
    merged4.to_excel(writer, sheet_name='Smartlogix Detailed Report', header=True, index=True)
    smart.to_excel(writer, sheet_name='Averages', header=True, index= True)

In [40]:
merged4.to_csv("fxz.csv", header=True, index=False)